In [52]:
import numpy as np
import pandas as pd
import math
import copy
from random import seed
from random import random
import matplotlib.pyplot as plt  # import matplotlib for plotting and visualization
import matplotlib
import PIL
import PIL.Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pathlib
from tensorflow.keras import layers
from struct import unpack
import os
import shutil
import random
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import argparse

In [69]:
## images path
orig_path= pathlib.Path(r'C:\Users\istra\Google Drive\Cours HEC\Maitrise\Semestre 2\Machine_Learning\TP GROUPE\book_covers2')
destin_path= pathlib.Path(r'C:\Users\istra\Google Drive\Cours HEC\Maitrise\Semestre 2\Machine_Learning\TP GROUPE\book_covers3')
image_count=len(list(orig_path.glob('*\*.jpg')))
print(image_count)

43325


In [58]:
img_augmentator = ImageDataGenerator(
	rotation_range=45,
	zoom_range=0.2,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
    vertical_flip=True,
	fill_mode="nearest")

Code to create random samples of equal size with data augmentation if not enough data 

In [59]:
subdirs = [subdir for subdir in orig_path.iterdir() if subdir.is_dir()]
for subdir in subdirs:
   subdir_len =  len(list(subdir.glob('*.jpg')))
   files = list(subdir.glob('*.jpg'))
   counter = 0
   sample_target = 4000
   sub_sample_size = min(sample_target,subdir_len)
   sub_sample = random.sample(files,sub_sample_size)
   while counter<sample_target:
       if counter <sub_sample_size:
            a = sub_sample[counter]

            label = os.path.split(subdir)[1]
            target =os.path.join(destin_path,label,os.path.basename(a))
            if not os.path.exists(str(destin_path)+"\\"+str(label)): 
                os.mkdir(str(destin_path)+"\\"+str(label))
            
            if not os.path.isfile(target):
                shutil.copy(a,target)
       else:
           image = load_img(random.choice(files))
           image = img_to_array(image)
           image = np.expand_dims(image, axis=0)
           destination =os.path.join(destin_path,label) 
           img_augmentator.flow(image, batch_size=1, save_to_dir=destination,
	save_prefix="augm_image", save_format="jpg").next()
       counter+=1
              

------------------------------------------------------

Creating a test set of 65 images

In [61]:
destin_path_Q= pathlib.Path(r'C:\Users\istra\Google Drive\Cours HEC\Maitrise\Semestre 2\Machine_Learning\TP GROUPE\Q')
subdirs = [subdir for subdir in orig_path.iterdir() if subdir.is_dir()]
for subdir in subdirs:
   subdir_len =  len(list(subdir.glob('*.jpg')))
   files = list(subdir.glob('*.jpg'))
   counter = 0
   sample_target = 5
   sub_sample = random.sample(files,sample_target)
   while counter<sample_target:
       a = sub_sample[counter]
       label = os.path.split(subdir)[1]
       target =os.path.join(destin_path_Q,str(label)+"_"+str(os.path.basename(a)))
       if not os.path.isfile(target):
                shutil.copy(a,target)
       counter+=1

Removing the images from the main test/validation folders

In [68]:
files = list(destin_path_Q.glob('*.jpg'))
for img_file in files:
   label,jpg_file = os.path.basename(img_file).split("_")
   target = os.path.join(orig_path,label,jpg_file)
   if os.path.isfile(target):
       os.remove(target)

In [74]:
test_genres = pd.read_csv("final_genres.csv")
imf =[os.path.basename(img_file).split("_")[1] for img_file in files]
test_genres = test_genres[test_genres["img"].isin(imf)]
test_genres.to_csv("test_sample.csv")

-------------------------
Check & Copy the selected files in sub-folders by genre

In [252]:
num_skipped = 0
num_copied = 0 
file_exists = 0
for index,book in final_genres_filtered.iterrows():
    genre = book["main_label"]
    fpath = str(orig_path) +"\\"+book["img"]
    target= str(destin_path)+"\\"+genre+"\\"+book["img"]
    if not os.path.exists(str(destin_path)+"\\"+genre): 
        os.mkdir(str(destin_path)+"\\"+genre)
    try:
        if os.path.isfile(target):
            file_exists+=1
        else:
            check_image(fpath)
    except:
        print("error, the following image was not copied",fpath)
        num_skipped+=1
    else:
        try:
            if not os.path.isfile(target):
                shutil.copy(fpath,target)
                num_copied +=1

        except IOError as e:
            print("Unable to copy file. %s" % e)
        except:
            print("Unexpected error:", sys.exc_info())
print("We skipped %d corrupt images" % num_skipped)
print("There were already %d files there" %file_exists)
print("We copied %d files" %num_copied)

\istra\img\images\images\44188.jpg
check error C:\Users\istra\img\images\images\44225.jpg
error, the following image was not copied C:\Users\istra\img\images\images\44225.jpg
check error C:\Users\istra\img\images\images\44241.jpg
error, the following image was not copied C:\Users\istra\img\images\images\44241.jpg
check error C:\Users\istra\img\images\images\44476.jpg
error, the following image was not copied C:\Users\istra\img\images\images\44476.jpg
check error C:\Users\istra\img\images\images\44641.jpg
error, the following image was not copied C:\Users\istra\img\images\images\44641.jpg
check error C:\Users\istra\img\images\images\44688.jpg
error, the following image was not copied C:\Users\istra\img\images\images\44688.jpg
check error C:\Users\istra\img\images\images\44733.jpg
error, the following image was not copied C:\Users\istra\img\images\images\44733.jpg
check error C:\Users\istra\img\images\images\44881.jpg
error, the following image was not copied C:\Users\istra\img\images\im

In [253]:
goodread["book_desc"].isna().sum()

1331